In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("spotify_millsongdata.csv")

In [6]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [8]:
df.shape

(57650, 4)

In [9]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [10]:
df.sample(5000).shape

(5000, 4)

In [11]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [12]:
df.head(10)

,artist,song,text
0,Michael W. Smith,Hey You It's Me,Hey you \r\nThings to do \r\nLive the life f...
1,The Monkees,I'm A Believer,I thought love was only true in fairy tales \...
2,Etta James,Strung Out,I will do whatever you say \r\nIt's got to be...
3,Coldplay,In My Place,"In my place, in my place \r\nWere lines that ..."
4,Whitesnake,She's A Woman,"She got everything I need, \r\nShe makes me s..."
5,Eminem,Despicable,"Fuck an intro man, let's go \r\nI come around..."
6,Doobie Brothers,Under The Spell,There's a stormy vision \r\nStars before my e...
7,Norah Jones,Are You Lonesome Tonight,Are you lonesome tonight ? \r\nDo you miss me...
8,Moody Blues,For My Lady,My boat sails stormy seas \r\nBattles oceans ...
9,Eddie Cochran,Willa Mae,"Oh Willa Mae, a-come a-shakin' by my way \r\n..."


In [13]:
df['text'][0]

"Hey you  \r\nThings to do  \r\nLive the life for faith and family  \r\nIt's tough  \r\nWith all that stuff  \r\nFilling your head  \r\nWe know  \r\nThe undertow  \r\nCould get us so we don't remember, but  \r\nLest we ever  \r\nLest we ever forget  \r\n  \r\nHey you, it's me  \r\nTrue lovers are we  \r\nChildren under the sun  \r\nTwo spirits in the name of One  \r\n  \r\nHey me  \r\nGot to be  \r\nPlaces to go and people there to see  \r\nNo space  \r\nFor back to base  \r\nAnd no time at all  \r\nWe talk  \r\nWalk the walk  \r\nRun the race  \r\nAnd climb the ladder, but  \r\nIf we ever  \r\nIf we ever should fall  \r\n  \r\nSometimes the smoke clears  \r\nAnd I see things how they are  \r\nI see how we have come so far\r\n\r\n"

In [161]:
# df = df.sample(5000)

In [14]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [15]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [17]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [18]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/udaym/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [23]:
similarity[0]

array([1.        , 0.02013963, 0.06904523, ..., 0.00741993, 0.01943562,
       0.0153391 ])

In [24]:

song_name = df.loc[10, 'song']
song_name

'All You Want When You Please'

In [25]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:11]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation(song_name)

['Brightest Star',
 'Hero',
 'Christmas (Baby Please Come Home)',
 'Please Make Up Your Mind',
 'All I Ever Wanted',
 'Still In Love',
 'Good Girls Go To Heaven',
 'Answer Me',
 'Christmas',
 'Love Was Here']

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))